In [10]:
import matplotlib.pyplot as plt
import numpy as np
import cPickle, h5py, os

from oc_ica import analysis
reload(analysis)

<module 'oc_ica.analysis' from '/home/jesse/Development/oc_ica/oc_ica/analysis.pyc'>

# HDF5 Storage

In [21]:
folder = '/home/jesse/Development/results/oc_ica'
a_file = 'a_array-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE.h5'
fit_files = ['comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-2.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-4.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-6.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COULOMB.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-COULOMB_F.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-RANDOM.h5',
             'comparison_mixtures-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE_models-RANDOM_F.h5']
models = [f.split('.')[-2].split('-')[-1] for f in fit_files]

In [27]:
colors = ['black', 'black', 'black', 'red', 'red', 'green', 'green']
fmts = ['-', '--', ':', '-', '--', '-', '--']
with h5py.File(os.path.join(folder, fit_files[0]), 'r') as f:
    lambdas = f['lambdas'].value
n_mixtures, n_sources = A_array.shape[2:]
n_iter = A_array.shape[1]

In [28]:
with h5py.File(os.path.join(folder, a_file), 'r') as f:
    A_array = f['A_array'].value
    A_priors = f['A_priors'].value

In [35]:
W_fits = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources, n_mixtures))
results = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, 3))
max_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources))
other_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources**2-n_sources))

for ii, f_name in enumerate(fit_files):
    with h5py.File(os.path.join(folder, f_name), 'r') as f:
        W_fits[:, ii] = np.squeeze(f['W_fits'])

(4, 1, 17, 40, 256, 128)
(4, 1, 17, 40, 256, 128)
(4, 1, 17, 40, 256, 128)
(4, 1, 17, 40, 256, 128)
(4, 1, 17, 40, 256, 128)
(4, 1, 17, 40, 256, 128)
(4, 1, 17, 40, 256, 128)


In [53]:
A_array.shape

(4, 40, 128, 256)

In [52]:
reload(analysis)
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                try:
                    A = A_array[ii, ll]
                    W = W_fits[ii, jj, kk, ll]
                    hd, (pd, ma, oa, mod) = analysis.recovery_statistics_AW(A, W)
                    results[ii, jj, kk, ll] = [hd, pd, mod]
                    max_angles[ii, jj, kk, ll] = ma
                    other_angles[ii, jj, kk, ll] = oa
                except ValueError:
                    print ii, jj, kk, ll

0 4 14 22
0 4 14 23
0 4 14 24
0 4 14 25
0 4 14 26
0 4 14 27
0 4 14 28
0 4 14 29
0 4 14 30
0 4 14 31
0 4 14 32
0 4 14 33
0 4 14 34
0 4 14 35
0 4 14 36
0 4 14 37
0 4 14 38
0 4 14 39
0 4 15 0
0 4 15 1
0 4 15 2
0 4 15 3
0 4 15 4
0 4 15 5
0 4 15 6
0 4 15 7
0 4 15 8
0 4 15 9
0 4 15 10
0 4 15 11
0 4 15 12
0 4 15 13
0 4 15 14
0 4 15 15
0 4 15 16
0 4 15 17
0 4 15 18
0 4 15 19
0 4 15 20
0 4 15 21
0 4 15 22
0 4 15 23
0 4 15 24
0 4 15 25
0 4 15 26
0 4 15 27
0 4 15 28
0 4 15 29
0 4 15 30
0 4 15 31
0 4 15 32
0 4 15 33
0 4 15 34
0 4 15 35
0 4 15 36
0 4 15 37
0 4 15 38
0 4 15 39
0 4 16 0
0 4 16 1
0 4 16 2
0 4 16 3
0 4 16 4
0 4 16 5
0 4 16 6
0 4 16 7
0 4 16 8
0 4 16 9
0 4 16 10
0 4 16 11
0 4 16 12
0 4 16 13
0 4 16 14
0 4 16 15
0 4 16 16
0 4 16 17
0 4 16 18
0 4 16 19
0 4 16 20
0 4 16 21
0 4 16 22
0 4 16 23
0 4 16 24
0 4 16 25
0 4 16 26
0 4 16 27
0 4 16 28
0 4 16 29
0 4 16 30
0 4 16 31
0 4 16 32
0 4 16 33
0 4 16 34
0 4 16 35
0 4 16 36
0 4 16 37
0 4 16 38
0 4 16 39
1 4 0 0
1 4 0 1
1 4 0 2
1 4 0 3
1 4 0 4


In [51]:
np.linalg.norm(W_fits[ii, jj, kk, ll], axis=1)

array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.00000001,  1.        ,  1.        ,
        1.        ,  1.        ,  0.99999999,  1.        ,  1.        ,
        0.99999999,  0.99999999,  0.99999999,  1.        ,  1.        ,
        1.00000001,  1.        ,  0.99999999,  0.99999999,  1.        ,
        1.        ,  1.        ,  1.        ,  0.99999999,  1.        ,
        0.99999999,  1.        ,  1.        ,  1.00000001,  1.00000001,
        1.        ,  1.        ,  1.00000001,  1.        ,  1.        ,
        1.        ,  1.        ,  0.99999999,  0.99999999,  1.00000001,
        0.99999999,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.00000001,  1.        ,  1.        ,  1.        ,
        1.00000001,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        0.99999999,  1.        ,  1.        ,  1.        ,  1.  

In [ ]:
f, axes = plt.subplots(len(A_priors), 4, figsize=(12, 12))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    (ad, ae, ap, aa) = a
    for jj, m in enumerate(models):
        if ii == 0:
            label = str(m)
            ad.legend(loc='best', prop={'size': 12})
        else:
            label = None
        ma = np.median(max_angles[ii, jj, :, :], axis=-1)
        oa = np.median(other_angles[ii, jj, :, :], axis=-1)
        da = (ma - oa).mean(axis=1)
        dist = results[ii, jj, :, :, 0].mean(axis=1)
        delta = results[ii, jj, :, :, 1].mean(axis=1)
        distp = results[ii, jj, :, :, 2].mean(axis=1)
        ad.semilogx(lambdas, dist, fmts[jj], label=label, c=colors[jj])
        ad.semilogx(lambdas, dist, '.', c=colors[jj])
        ae.semilogx(lambdas, delta, fmts[jj], label=label, c=colors[jj])
        ae.semilogx(lambdas, delta, '.', c=colors[jj])
        ap.semilogx(lambdas, distp, fmts[jj], label=label, c=colors[jj])
        ap.semilogx(lambdas, distp, '.', c=colors[jj])
        aa.semilogx(lambdas, da, fmts[jj], label=label, c=colors[jj])
        aa.semilogx(lambdas, da, '.', c=colors[jj])

    ad.set_title(p)
    ad.set_xlabel('Lambda')
    ad.set_ylabel('Hellinger Distance')
    ae.set_title(p)
    ae.set_xlabel('Lambda')
    ae.set_ylabel('Permutation Delta')
    ap.set_title(p)
    ap.set_xlabel('Lambda')
    ap.set_ylabel('Hellinger Distance Max vs. Other')
    aa.set_title(p)
    aa.set_xlabel('Lambda')
    aa.set_ylabel('Difference in Median Max vs. Other')
f.tight_layout()
#    plt.savefig(p+'.pdf')
plt.show()
plt.close()

In [ ]:
f, axes = plt.subplots(len(A_priors), len(models), figsize=(24, 8))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    for jj, (m, ax) in enumerate(zip(models, a)):
        ma = max_angles[ii, jj, 4, :]
        oa = other_angles[ii, jj, 4, :]
        md, bins = analysis.angle_histogram(ma)
        od, bins = analysis.angle_histogram(oa)
        ax.semilogy(bins[:-1], md, drawstyle='steps', color='yellow', lw=1.5)
        ax.semilogy(bins[:-1], od, drawstyle='steps', color='green', lw=1.5)

        ax.set_title(p)
        ax.set_xlabel('Angle')
        ax.set_ylabel(str(m))
        ax.set_xticks([0, 45, 90])
        ax.set_yticks([1e-6, 1e-3, 1e0])
f.tight_layout()
#    plt.savefig(p+'.pdf')
plt.show()
plt.close()

# Pickle Storage

In [3]:
fname = ''
with open(fname, 'r') as f:
    data = cPickle.load(f)

In [22]:
A_dict, W_dict, W_fits, min_ks, results = data
n_mixtures, n_sources = A_dict[A_dict.keys()[0]][0].shape
n_iter = W_fits.shape[3]
results = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, 3))
max_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources))
other_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources**2-n_sources))

In [23]:
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        A = A_dict[p][jj]
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                W = W_fits[ii, jj, kk, ll]
                hd, (pd, ma, oa, mod) = analysis.recovery_statistics_AW(A, W)
                results[ii, jj, kk, ll] = [hd, pd, mod]
                max_angles[ii, jj, kk, ll] = ma
                other_angles[ii, jj, kk, ll] = oa